In [161]:
import time
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
import requests

from time import sleep
import random

In [162]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service


service = Service(executable_path="geckodriver.exe")  # Provide the path to the geckodriver executable
driver = webdriver.Firefox(service=service)
driver.maximize_window()


In [77]:
# Open the Mayo Clinic Minnesota webpage
website = "https://www.mayoclinic.org/patient-visitor-guide/minnesota"
driver.get(website)

# Extracts the contents of specific sections from the page
contents = driver.find_elements(by='xpath', value='//div[@class="contentbox"]')

# Get the name and overview from the content boxes
name = contents[0].text  # First content box typically contains the name
overview = contents[1].text  # Second content box typically contains the overview

# Find the booking section, extract the href link
booking = driver.find_element(by='xpath', value='//div[@class="contentbox mc-promo-tile"]')
link_elements = booking.find_element(By.XPATH, value='.//a')  # Find the booking link within the promo tile
link_herf = link_elements.get_attribute("href")  # Extract the URL of the booking link

# Close the popup by clicking on the button
driver.find_element(by='xpath', value='//button[@class="cmp-button cmp-button__button "]').click()

# Navigate to a section by clicking on the identified button
driver.find_element(by='xpath', value='//a[@id="button-c80143308f"]').click()

# Extract additional info from the content box on the new page
info = driver.find_element(by='xpath', value='//div[@class="list content-within"]').text

# Go back to the previous page
driver.back()

# Close another popup by clicking the button
driver.find_element(by='xpath', value='//button[@class="cmp-button cmp-button__button "]').click()

# Navigate to another section by clicking a different button
driver.find_element(by='xpath', value='//a[@id="button-0ccc721927"]').click()

# Click on a specific alphabetical filter for doctor listings (e.g., for doctors with last names starting with 'A')
driver.find_element(by='xpath', value='//a[@id="et_alphaListFilter_A_e83fa9a8-2811-4ed4-bed2-454b32845156"]').click()

# Initialize an empty list to store doctor data
doctors_data = []

# Define a helper function to extract text, returns "NA" if element is not found
def get_text_or_na(element, by, value):
    try:
        return driver.find_element(by=by, value=value).text.strip()
    except:
        return "NA"

# Locate the doctor elements from the list of search results
doctor_elements = driver.find_elements(By.XPATH, '//ol[@class="result-items"]/li')

# Loop through each doctor element to scrape relevant details
for doctor in doctor_elements:
    doctor_data = {}  # Initialize an empty dictionary for each doctor

    # Extract the doctor's name
    try:
        name_element = doctor.find_element(By.CSS_SELECTOR, 'h4 a')
        doctor_data['name'] = name_element.text.strip()
    except:
        doctor_data['name'] = "NA"

    # Extract the doctor's roles (e.g., specialties)
    try:
        role_element = doctor.find_element(By.XPATH, './/ol[@class="speciality"]')
        roles = role_element.text.strip().split('\n')  # Split roles by newline
        doctor_data['roles'] = [role.strip() for role in roles if role.strip()]  # Remove any empty role entries
    except:
        doctor_data['roles'] = ["NA"]

    # Extract the doctor's location
    try:
        location_element = doctor.find_element(By.XPATH, './/ol[@class="location"]')
        doctor_data['location'] = location_element.text.strip()
    except:
        doctor_data['location'] = "NA"

    # Extract areas of focus (e.g., medical focus areas)
    try:
        areas_element = doctor.find_element(By.XPATH, './/div[@class="ellipsis-container"]')
        areas = areas_element.text.strip().split('\n')  # Split areas by newline
        areas = [area.strip() for area in areas if area.strip()]  # Remove any empty areas
        # Clean and filter out unnecessary text (e.g., "Show more areas of focus")
        doctor_data['areas_of_focus'] = [re.sub(r'Show more areas of focus|for [\w\s,]+', '', area).strip() for area in areas]
    except:
        doctor_data['areas_of_focus'] = ["NA"]

    # Append the doctor's data to the list of doctors
    doctors_data.append(doctor_data)

# Print the scraped data in JSON format
print(json.dumps(doctors_data, indent=4))

# Optionally, save the data to a file
with open('doctors_data.json', 'w') as file:
    json.dump(doctors_data, file, indent=4)

# Close the browser session once scraping is completed
driver.quit()


[
    {
        "name": "Chris A. Aakre, M.D.",
        "roles": [
            "Internist"
        ],
        "location": "Rochester, MN",
        "areas_of_focus": [
            "Integrative medicine"
        ]
    },
    {
        "name": "Ejigayehu G. Abate, M.D.",
        "roles": [
            "Internist",
            "Endocrinologist"
        ],
        "location": "Jacksonville, FL",
        "areas_of_focus": [
            "Metabolic bone disease, Osteoporosis, Hyperparathyroidism, Hypoparathyroidism"
        ]
    },
    {
        "name": "Charles F. Abboud, M.B.",
        "roles": [
            "NA"
        ],
        "location": "Rochester, MN",
        "areas_of_focus": [
            "NA"
        ]
    },
    {
        "name": "Arney S. Abcejo, M.D.",
        "roles": [
            "Anesthesiologist"
        ],
        "location": "Rochester, MN",
        "areas_of_focus": [
            "NA"
        ]
    },
    {
        "name": "Ahmed D. Abdalrhim, M.D.",
        "roles": 

In [18]:
departments_list = info.split("\n")


departments_list = [dept.strip() for dept in departments_list if dept.strip() and len(dept.strip()) > 1]


print(departments_list)

['All Departments', 'Advanced Care at Home', 'Aerospace Medicine in Rochester, Minnesota', 'Allergic Diseases', 'Anesthesiology and Perioperative Medicine', 'Audiology', 'Bariatric Center in Florida', 'Brain Tumor Program', 'Breast and Melanoma Surgical Oncology in Rochester', 'Breast Clinic', 'Breast Surgical Oncology in Jacksonville', 'Cardiovascular Medicine', 'Cardiovascular Medicine in Jacksonville', 'Cardiovascular Medicine in Phoenix', 'Cardiovascular Medicine in Rochester', 'Cardiovascular Surgery', 'Center for Aesthetic Medicine & Surgery in Rochester', 'Center for Individualized Medicine', "Children's Center", 'Clinical Genomics', 'Colon and Rectal Surgery', 'Mayo Clinic Comprehensive Cancer Center', 'CAR-T Cell Therapy Program', 'Proton Beam Therapy Program', 'Mayo Clinic 365 Concierge Medicine in Florida', 'Cosmetic Center in Arizona', 'Critical Care', 'Dental Specialties', 'Dermatology', 'Early Onset and Hereditary GI Cancers Program', 'Emergency Medicine', 'Endocrine Surg

In [126]:
searchterm='A'



In [127]:
# Set the Cleveland Clinic website URL and navigate to it
website = "https://my.clevelandclinic.org/"
driver.get(website)

# Extract the clinic's name from the header logo
name = driver.find_element(by='xpath', value='//span[@class="header__logo"]').text

# Extract the overview from the spotlight section
overview = driver.find_element(by='xpath', value='//div[@class="spotlight-panel__container js-spotlight-panel__container container"]').text

# Locate and interact with the story panel (although nothing is extracted from it here)
driver.find_element(by='xpath', value='//div[@class="story-panel__card-content story-panel__no-selection "]')

# Click on a button within the story panel to navigate to the next page or section
driver.find_element(by='xpath', value='//a[@class="story-panel__button button"]').click()
time.sleep(2)  # Wait for the page to load

# Find the search input field and enter the search term (e.g., a doctor's name or specialty)
search = driver.find_element(by='xpath', value='//input[@class="js-search-field field__full-width field__big js-clearable"]')
search.send_keys(searchterm)  # 'searchterm' should be defined as a variable earlier in your code
time.sleep(2)  # Allow time for the search results to load

# Locate the search results container
hit = driver.find_element(by='xpath', value='//div[@id="hits"]')

# Extract the list of doctors from the search results
doctors = hit.find_elements(by='xpath', value='//div[@class="list-item-search LEGACY-list-pad-bottom provider-search__result-wrapper js-provider-search__result-wrapper"]')

# Initialize an empty list to store doctor information
doctor_list = []

# Loop through each doctor in the search results
for doctor in doctors:
    doctor_info = {}  # Dictionary to store details of each doctor

    # Extract the doctor's name
    name = doctor.find_element(by='xpath', value='.//h2[@class="list-item-search__title lede"]').text
    doctor_info['NAME'] = name  # Store the doctor's name in the dictionary
    
    # Locate the left section containing department and location information
    left = doctor.find_element(by='xpath', value='.//div[@class="list-item-search__left"]')

    # Extract the department and location by selecting the appropriate unordered list (ul) elements
    d_l = left.find_elements(by='xpath', value='.//ul[@class="list-item-search__list"]')

    # Check if the department and location elements are present
    if len(d_l) >= 2:
        doctor_info['department'] = d_l[0].text  # First list contains the department
        doctor_info['location'] = d_l[1].text  # Second list contains the location
    
    # Append the doctor information to the list of doctors
    doctor_list.append(doctor_info)

# Optionally, you can print or save the doctor information
print(json.dumps(doctor_list, indent=4))  # Print the list in JSON format

# Clean up or save the data to a file
with open('cleveland_clinic_doctors.json', 'w') as file:
    json.dump(doctor_list, file, indent=4)

# Close the browser once the scraping is done
driver.quit()


In [131]:
doctor_info

{'NAME': 'Mamoun Abdoh, MD',
 'department': 'Pulmonary Medicine, Critical Care Medicine',
 'location': 'Hillcrest Hospital'}

In [132]:
doctor_list

[{'NAME': 'Ksenia Aaron, MD',
  'department': 'Head and Neck Institute',
  'location': 'Twinsburg Family Health & Surgery Center, Cleveland Clinic Main Campus'},
 {'NAME': 'Sagiv Aaron, MD',
  'department': 'Primary Care Pediatrics',
  'location': 'Independence Family Hlth Ctr'},
 {'NAME': 'Katarina Abadie, MD',
  'department': 'Family Medicine',
  'location': 'Solon Family Health Center'},
 {'NAME': 'Justin Abbatemarco, MD',
  'department': 'Mellen Center',
  'location': 'Cleveland Clinic Main Campus, Twinsburg Family Health & Surgery Center'},
 {'NAME': 'Aza Abdalla, MD',
  'department': 'Transplantation Center',
  'location': 'Weston Hospital'},
 {'NAME': 'Sawsan Abdel-Razig, MD',
  'department': 'International Operations',
  'location': 'Abu Dhabi'},
 {'NAME': 'Sarah AbdelFattah, MD',
  'department': 'Anesthesiology',
  'location': 'Weston Hospital'},
 {'NAME': 'Yaser Abdelhamid, ND, LAc',
  'department': 'Center for Integrative Medicine',
  'location': 'Avon - Richard E. Jacobs He

In [133]:
doctor_info_json = json.dumps(doctor_list, indent=4)

# Print the JSON to verify
print(doctor_info_json)

# Optionally, write the JSON to a file
with open('doctor_info_celeveland.json', 'w') as json_file:
    json_file.write(doctor_info_json)

[
    {
        "NAME": "Ksenia Aaron, MD",
        "department": "Head and Neck Institute",
        "location": "Twinsburg Family Health & Surgery Center, Cleveland Clinic Main Campus"
    },
    {
        "NAME": "Sagiv Aaron, MD",
        "department": "Primary Care Pediatrics",
        "location": "Independence Family Hlth Ctr"
    },
    {
        "NAME": "Katarina Abadie, MD",
        "department": "Family Medicine",
        "location": "Solon Family Health Center"
    },
    {
        "NAME": "Justin Abbatemarco, MD",
        "department": "Mellen Center",
        "location": "Cleveland Clinic Main Campus, Twinsburg Family Health & Surgery Center"
    },
    {
        "NAME": "Aza Abdalla, MD",
        "department": "Transplantation Center",
        "location": "Weston Hospital"
    },
    {
        "NAME": "Sawsan Abdel-Razig, MD",
        "department": "International Operations",
        "location": "Abu Dhabi"
    },
    {
        "NAME": "Sarah AbdelFattah, MD",
        "dep

In [175]:
# Set the Massachusetts General Hospital website URL and navigate to it
website = "https://www.massgeneral.org/"
driver.get(website)

# Extract the overview content from a specific section of the page
overview = driver.find_element(by='xpath', value='//div[@class="iw_component" and @id="iw_comp1563765926222"]').text

# Locate and interact with the main hero section (though not extracting data from it)
driver.find_element(by='xpath', value='//div[@class="hero-content"]')

# Locate the Call-to-Action (CTA) buttons section on the page (though not extracting any data here either)
driver.find_element(by='xpath', value='//div[@class="ctas"]')

# Click on the "Find a doctor" button to navigate to the doctor search page
driver.find_element(by='xpath', value='//a[@aria-label="Find a doctor"]').click()

# Wait for the page to load
time.sleep(2)

# Locate the section that contains the doctor listings
box = driver.find_element(by='xpath', value='//div[@id="sectionk0kwc6wr" and @class="container-fluid iw_section"]')

# Extract a list of doctor profiles from the search results
doctor = box.find_elements(by='xpath', value='//div[@class="listing-item listing-item--profile"]')

# Initialize an empty list to store doctor information
doctor_list_mass = []

# Loop through each doctor profile and extract relevant information
for doc in doctor:
    doc_info_mass = {}  # Dictionary to store doctor details
    
    # Extract the doctor's name from the h4 tag
    name = doc.find_element(by='xpath', value='.//h4[@class="listing-item__title"]').text
    doc_info_mass['name'] = name  # Store the doctor's name in the dictionary
    
    # Extract the doctor's location or additional information from the meta section
    location = doc.find_element(by='xpath', value='.//div[@class="listing-item__meta"]').text
    doc_info_mass['info'] = location  # Store the doctor's location in the dictionary
    
    # Append the extracted doctor information to the list
    doctor_list_mass.append(doc_info_mass)

# Optionally, print the extracted data
print(json.dumps(doctor_list_mass, indent=4))

# Optionally, save the doctor data to a file
with open('massgeneral_doctors.json', 'w') as file:
    json.dump(doctor_list_mass, file, indent=4)

# Clean up and close the browser
driver.quit()


In [176]:
overview

"In the delivery of our care, through our research and within our communities, Mass General is committed to the well-being of our patients locally and globally.\nWe bridge innovation science with state-of-the-art clinical medicine.\nLearn more\nDiscover the ways we're training the next generation of doctors and researchers.\nLearn more\nLearn about Mass General's service within greater Boston and around the world.\nLearn more"

In [177]:
doctor

[<selenium.webdriver.remote.webelement.WebElement (session="3ebb6d45-cb5f-4bfb-a5bb-bec52852aceb", element="ef58ed93-cb77-43f3-9ffd-686e8925f896")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3ebb6d45-cb5f-4bfb-a5bb-bec52852aceb", element="01d67083-a16a-428a-8d74-973c2b2b38d5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3ebb6d45-cb5f-4bfb-a5bb-bec52852aceb", element="4b6c8d61-23dc-4117-8121-5b4f1e204884")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3ebb6d45-cb5f-4bfb-a5bb-bec52852aceb", element="de16bd58-726c-4d6a-9567-3f8e3858ddb5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3ebb6d45-cb5f-4bfb-a5bb-bec52852aceb", element="71e890cc-2c64-4267-8912-3fa05917a64c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3ebb6d45-cb5f-4bfb-a5bb-bec52852aceb", element="7d7f5394-60fc-422c-9300-577b4e0c2e17")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3ebb6d45-cb5f-4bfb-a5bb-bec52852aceb", element

In [178]:
doctor_list_mass

[{'name': 'Emily Aaronson, MD', 'info': 'Location: Boston'},
 {'name': 'Matthew Abadir, MD', 'info': 'Locations: Boston, Salem'},
 {'name': 'Lauren Abbate, MD',
  'info': 'Primary Care Physician\nLocation: Boston'},
 {'name': 'Gerald Abbott, MD', 'info': 'Radiologist\nLocation: Boston'},
 {'name': 'Shadi Abdar Esfahani, MD, MPH',
  'info': 'Radiologist\nLocation: Boston'},
 {'name': 'Sama Abdul-Aziz, DMD', 'info': 'Location: Boston'},
 {'name': 'Dunia Abdul-Aziz, MD', 'info': ''},
 {'name': 'Madelaine Abel, PhD', 'info': ''},
 {'name': 'Rachel Abelman, MD',
  'info': 'Medical Oncologist\nLocation: Boston'},
 {'name': 'John Ablon, PhD', 'info': 'Location: Boston'}]

In [179]:
doctor_info_json = json.dumps(doctor_list_mass, indent=4)

# Print the JSON to verify
print(doctor_info_json)

# Optionally, write the JSON to a file
with open('doctor_info_Massachusetts.json', 'w') as json_file:
    json_file.write(doctor_info_json)

[
    {
        "name": "Emily Aaronson, MD",
        "info": "Location: Boston"
    },
    {
        "name": "Matthew Abadir, MD",
        "info": "Locations: Boston, Salem"
    },
    {
        "name": "Lauren Abbate, MD",
        "info": "Primary Care Physician\nLocation: Boston"
    },
    {
        "name": "Gerald Abbott, MD",
        "info": "Radiologist\nLocation: Boston"
    },
    {
        "name": "Shadi Abdar Esfahani, MD, MPH",
        "info": "Radiologist\nLocation: Boston"
    },
    {
        "name": "Sama Abdul-Aziz, DMD",
        "info": "Location: Boston"
    },
    {
        "name": "Dunia Abdul-Aziz, MD",
        "info": ""
    },
    {
        "name": "Madelaine Abel, PhD",
        "info": ""
    },
    {
        "name": "Rachel Abelman, MD",
        "info": "Medical Oncologist\nLocation: Boston"
    },
    {
        "name": "John Ablon, PhD",
        "info": "Location: Boston"
    }
]
